In [1]:
import ipynb.fs.full.preprocess_eda as preprocess

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The length of the training data:  7500
The length of the testing data:  4500
The length of the validation data:  3000


In [2]:
from deep_translator import GoogleTranslator

In [ ]:
import os

In [38]:
PATH = r"D:\MScDataScience\7.Data_Science_Project\SourceCode\augment_data\Back_Translation"
traget_languages = ["hi", "fr", "de", "es"]

In [49]:
class back_translate():
    def __init__(self, traget_languages, path, data):
        self.traget_languages = traget_languages
        self.path = path
        self.data = data

    def google_translate(self, original_text, traget):
        translated_text = GoogleTranslator(source="en", target=traget).translate_batch(original_text)
        return GoogleTranslator(source=traget, target="en").translate_batch(translated_text)
    
    def augment(self):
        original_text = self.data.iloc[:, 0].to_list()
        labels = self.data.iloc[:, 1]
        for traget in self.traget_languages:
            print(f"[INFO] Augmenting data using language {traget} ...")
            preprocess.pd.DataFrame({
                "Query": self.google_translate(original_text, traget),
                "Intent": labels
            }).to_csv(os.path.join(self.path, 
                                   f"back_translation_augment_data_{traget}.csv"))
            print("\t Augmented and saved to the disk.")

In [50]:
translate_augment = back_translate(traget_languages, PATH, preprocess.train_df)

In [ ]:
translate_augment.augment()

[INFO] Augmenting data using language hi ...
